In [ ]:
import ee
import geemap
ee.Authenticate()

In [ ]:
ee.Initialize(project='sound-bit-482704-n1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Satellite-Imagery/test2(test(1)).csv")
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [ ]:
df.shape

(5404, 20)

In [ ]:
df.columns = [
    "id",
    "date",
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "zipcode",
    "latitude",
    "longitude",
    "sqft_living15",
    "sqft_lot15"
]
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,latitude,longitude,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [ ]:
from datetime import datetime

def parse_date(date_str):
  return datetime.strptime(date_str, "%Y%m%dT%H%M%S")


In [ ]:
def get_collection_by_date(date, point):
  start = ee.Date(date).advance(-4, 'month')
  end = ee.Date(date).advance(4, 'month')


  year = date.year


  if year <= 2011:
    col = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    bands = ['SR_B3','SR_B2','SR_B1']
    scale = 30
  elif year <= 2013:
    col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    bands = ['SR_B3','SR_B2','SR_B1']
    scale = 30
  elif year <= 2015:
    col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    bands = ['SR_B4','SR_B3','SR_B2']
    scale = 30
  else:
    col = ee.ImageCollection('COPERNICUS/S2_SR')
    bands = ['B4','B3','B2']
    scale = 10


  col = col.filterBounds(point).filterDate(start, end)
  return col, bands, scale



In [ ]:
import os


OUT_DIR = '/content/drive/MyDrive/images_cdc_test'
os.makedirs(OUT_DIR, exist_ok=True)

CHIP_PX = 416


def export_cnn_chip(row):
    img_id = row['id']
    lat = row['latitude']
    lon = row['longitude']
    date = parse_date(row['date'])

    point = ee.Geometry.Point(lon, lat)
    col, bands, scale = get_collection_by_date(date, point)

    if col.size().getInfo() == 0:
        print(f"⚠️ {img_id}: no imagery")
        return False

    image = col.sort('CLOUD_COVER').first().select(bands)

    half = (CHIP_PX * scale) // 2
    region = point.buffer(half).bounds()

    out_path = os.path.join(OUT_DIR, f"{img_id}.tif")

    geemap.ee_export_image(
        image,
        out_path,
        region=region,
        scale=scale,
        file_per_band=False
    )

    print(f"✅ {img_id} | {scale}m | {date.date()}")
    return True







In [ ]:
saved_count = 0
total = len(df)

for i, (_, row) in enumerate(df.iterrows(), start=1):
    success = export_cnn_chip(row)
    if success:
        saved_count += 1

    print(f"Progress: {i}/{total} | Saved: {saved_count}")


Streaming output truncated to the last 5000 lines.
✅ 8898700440 | 30m | 2014-11-24
Progress: 1861/5404 | Saved: 1861
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/723049197.tif
✅ 723049197 | 30m | 2014-06-27
Progress: 1862/5404 | Saved: 1862
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/2019200220.tif
✅ 2019200220 | 30m | 2015-02-26
Progress: 1863/5404 | Saved: 1863
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/2926049408.tif
✅ 2926049408 | 30m | 2014-10-09
Progress: 1864/5404 | Saved: 1864
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/4233600190.tif
✅ 4233600190 | 30m | 2015-03-16
Progress: 1865/5404 | Saved: 1865
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/9268851860.tif
✅ 9268851860 | 30m | 2014-09-18
Progress: 1866/5404 | Saved: 1866
Generating URL 

Streaming output truncated to the last 5000 lines.
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/3444120130.tif
✅ 3444120130 | 30m | 2015-03-27
Progress: 4512/5404 | Saved: 4512
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/5256500025.tif
✅ 5256500025 | 30m | 2014-08-27
Progress: 4513/5404 | Saved: 4513
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/4022902715.tif
✅ 4022902715 | 30m | 2015-04-03
Progress: 4514/5404 | Saved: 4514
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/871001500.tif
✅ 871001500 | 30m | 2014-06-23
Progress: 4515/5404 | Saved: 4515
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/2329700440.tif
✅ 2329700440 | 30m | 2014-10-27
Progress: 4516/5404 | Saved: 4516
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/1773100

Data downloaded to /content/drive/MyDrive/images_cdc_test/868000175.tif
✅ 868000175 | 30m | 2014-10-01
Progress: 5345/5404 | Saved: 5345
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/1446401290.tif
✅ 1446401290 | 30m | 2014-10-30
Progress: 5346/5404 | Saved: 5346
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/9109000050.tif
✅ 9109000050 | 30m | 2014-07-09
Progress: 5347/5404 | Saved: 5347
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/8856004327.tif
✅ 8856004327 | 30m | 2014-05-09
Progress: 5348/5404 | Saved: 5348
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/2329800110.tif
✅ 2329800110 | 30m | 2015-01-28
Progress: 5349/5404 | Saved: 5349
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/images_cdc_test/3031200230.tif
✅ 3031200230 | 30m | 2015-02-26
Progress: 5350/5404 | Saved